### Objective
The objective of this study is to develop a vision-based vehicle detection system for intelligent driving. *The paper proposes a vision-based vehicle detection system for intelligent driving. By using low-quality images from a monocular camera mounted on the front of a vehicle, the system aims to accurately detect and avoid obstacles (like other vehicles) to prevent accidents.*

### Methodology
The detection system is built around a CNN, which can efficiently extract features from input images and provide accurate recognitionof vehicles in real-time. The paper explores different CNN architectures and compares their performance, demonstrating that thisapproach yields high accuracy in vehicle detection under varying conditions.

### Key Components:
- CNN is utilized for feature extraction, classification, and vehicle detection from road images.
- The system involves a multi-layer CNN structure with convolution, pooling, and fully connected layers.
- The CNN-based system’s performance is compared to other models, such as Support Vector Machines (SVM) and Fully Connected Neural Networks (FFNN), with CNN outperforming them.

##### 1. Import Libraries

In [4]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split

/Users/macbook/Documents/GitHub/my_vehicle_detection/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


##### the CNN model has three convolutional layers followed by pooling layers and dense layers

### Load and Preprocess Images

In [8]:
# Set image dimensions
IMG_WIDTH, IMG_HEIGHT = 64, 64

# Load vehicle images and label them as 1
vehicle_paths = sorted(glob.glob('./VehicleImage/vehicles/*/*.png'))
vehicles = [cv2.resize(cv2.imread(img), (IMG_WIDTH, IMG_HEIGHT)) for img in vehicle_paths]
vehicle_labels = [1] * len(vehicles)

# Load non-vehicle images and label them as 0
nonvehicle_paths = sorted(glob.glob('./VehicleImage/non-vehicles/*/*.png'))
nonvehicles = [cv2.resize(cv2.imread(img), (IMG_WIDTH, IMG_HEIGHT)) for img in nonvehicle_paths]
nonvehicle_labels = [0] * len(nonvehicles)

# Combine data and labels
X = np.array(vehicles + nonvehicles)
y = np.array(vehicle_labels + nonvehicle_labels)

### 1. Split the Data

In [9]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0

# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)    

### 1.5 Split the Dataset into Training, Validation, and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Initial split: Training and temporary set (combined validation + test)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Second split: Validation and training
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)
# 0.1765 * 0.85 ~= 0.15, so we end up with 70% train, 15% val, 15% test

# Normalize the images
X_train, X_val, X_test = X_train / 255.0, X_val / 255.0, X_test / 255.0

# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

#### Define the CNN Model


In [5]:
def create_cnn_model(input_shape=(64, 64, 3)):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten and Fully Connected Layers
    model.add(Flatten())
    
    model.add(Dense(16, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    
    # Output Layer: 2 neurons (binary classification for "vehicle" or "no vehicle")
    model.add(Dense(2, activation='softmax'))
    
    return model

# Instantiate the model
model = create_cnn_model()
model.summary()

/Users/macbook/Documents/GitHub/my_vehicle_detection/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-29 17:17:01.945820: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-10-29 17:17:01.945879: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-10-29 17:17:01.945885: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-10-29 17:17:01.946156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-29 17:17:01.946172: I tensorflow/core/common_runtime/pluggable_dev

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │        73,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 176,658 (690.07 KB)

 Trainable params: 176,658 (690.07 KB)

 Non-trainable params: 0 (0.00 B)

### Different CNN Design

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
from tensorflow.keras.optimizers import Adam

def create_cnn_model_arch2(input_shape=(64, 64, 3)):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3)))
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 3
    model.add(Conv2D(128, (3, 3)))
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten and Fully Connected Layers
    model.add(Flatten())
    
    model.add(Dense(16))
    model.add(ReLU())
    model.add(Dense(64))
    model.add(ReLU())
    model.add(Dense(128))
    model.add(ReLU())
    
    # Output Layer: 2 neurons (binary classification for "vehicle" or "no vehicle")
    model.add(Dense(2, activation='softmax'))
    
    return model

# Instantiate the model
model_arch2 = create_cnn_model_arch2()
model_arch2.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_18 (ReLU)                 │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_19 (ReLU)                 │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_20 (ReLU)                 │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │        73,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 176,658 (690.07 KB)

 Trainable params: 176,658 (690.07 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# compile model_arch2
model_arch2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

##### 3. Compile the Model

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# # Define paths for train and validation directories
# train_dir = '/path/to/train'  # Replace with your actual path
# validation_dir = '/path/to/validation'  # Replace with your actual path

# # Data Augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1.0/255,
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# )

# validation_datagen = ImageDataGenerator(rescale=1.0/255)

# # Load Data
# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(64, 64),
#     batch_size=32,
#     class_mode='binary'
# )

# validation_generator = validation_datagen.flow_from_directory(
#     validation_dir,
#     target_size=(64, 64),
#     batch_size=32,
#     class_mode='binary'
# )

##### 5. Train the Model

In [22]:
# Ensure the model is defined by running the cell where the model is created and compiled
history = model_arch2.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.7546 - loss: 0.4554 - val_accuracy: 0.9556 - val_loss: 0.1202
Epoch 2/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.9443 - loss: 0.1507 - val_accuracy: 0.9768 - val_loss: 0.0858
Epoch 3/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9687 - loss: 0.0883 - val_accuracy: 0.9638 - val_loss: 0.1122
Epoch 4/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9818 - loss: 0.0559 - val_accuracy: 0.9775 - val_loss: 0.0654
Epoch 5/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9850 - loss: 0.0431 - val_accuracy: 0.9747 - val_loss: 0.0712
Epoch 6/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9807 - loss: 0.0504 - val_accuracy: 0.9843 - val_loss: 0.0381
Epoch 7/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9905 - loss: 0.0297 - val_accuracy: 0.9823 - val_loss: 0.0532
Epoch 8/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9892 - loss: 0.0292 - val_acc

KeyboardInterrupt: 

##### 6.  Evaluate the Model

In [ ]:
# Evaluate model performance on validation data
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

##### 7. Make Predictions

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_vehicle(model, img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if np.argmax(prediction) == 0:
        print("No Vehicle Detected")
    else:
        print("Vehicle Detected")

# Test on a new image
predict_vehicle(model, '/path/to/test_image.jpg')

### Paper CNN Architecture

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model_arch3 = models.Sequential()

# Input layer
model_arch3.add(layers.Input(shape=(64, 64, 3)))

# Hidden layer 1
model_arch3.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model_arch3.add(layers.MaxPooling2D((2, 2)))

# Hidden layer 2
model_arch3.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model_arch3.add(layers.MaxPooling2D((2, 2)))

# Hidden layer 3
model_arch3.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model_arch3.add(layers.MaxPooling2D((2, 2)))

# Classification layer
model_arch3.add(layers.Flatten())
model_arch3.add(layers.Dense(16, activation='relu'))
model_arch3.add(layers.Dense(64, activation='relu'))
model_arch3.add(layers.Dense(128, activation='relu'))
model_arch3.add(layers.Dense(2, activation='softmax'))

# Compile the model_arch3
model_arch3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model_arch3
model_arch3.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 16)             │       131,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 234,002 (914.07 KB)

 Trainable params: 234,002 (914.07 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# compile the model
model_arch3.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# fit thte model
history = model_arch3.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.7940 - loss: 0.4333 - val_accuracy: 0.9556 - val_loss: 0.1334
Epoch 2/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9585 - loss: 0.1159 - val_accuracy: 0.9652 - val_loss: 0.1009
Epoch 3/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9781 - loss: 0.0646 - val_accuracy: 0.9782 - val_loss: 0.0702
Epoch 4/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9823 - loss: 0.0507 - val_accuracy: 0.9823 - val_loss: 0.0477
Epoch 5/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9851 - loss: 0.0467 - val_accuracy: 0.9727 - val_loss: 0.0806
Epoch 6/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9852 - loss: 0.0443 - val_accuracy: 0.9898 - val_loss: 0.0332
Epoch 7/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9942 - loss: 0.0188 - val_accuracy: 0.9816 - val_loss: 0.0601
Epoch 8/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9934 - loss: 0.0222 - val_accu

In [27]:
# Evaluate model performance on validation data
loss, accuracy = model_arch3.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9883 - loss: 0.1088
Validation Loss: 0.09181050211191177
Validation Accuracy: 0.9904437065124512


## analyze the model’s performance in detail

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Step 2: Make Predictions on the Test Set


In [ ]:
# Predict probabilities
y_pred_probs = model.predict(X_test)
# Convert probabilities to binary predictions (0 or 1 for non-vehicle or vehicle)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)  # True labels

Step 3: Calculate and Interpret Metrics

In [ ]:
# 1. Accuracy: Measures the proportion of correct predictions (both positive and negative) out of the total predictions.
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")


- Interpretation:
    - Accuracy shows the overall correctness but may be misleading if there is class imbalance.

In [ ]:
# 2. Precision: Measures the proportion of true positive predictions out of all positive predictions. High precision indicates that the model is good at minimizing false positives.
precision = precision_score(y_true, y_pred)
print(f"Precision: {precision}")



- Interpretation: 
    - Precision is useful when false positives are costly. In vehicle detection, low precision would mean the model often misclassifies non-vehicles as vehicles.

In [ ]:
# 3. Recall (Sensitivity): Measures the proportion of true positive predictions out of all actual positives. High recall indicates that the model is good at minimizing false negatives.
recall = recall_score(y_true, y_pred)
print(f"Recall: {recall}")


- Interpretation: 
  - Recall is useful when false negatives are costly. In vehicle detection, low recall would mean the model misses vehicles on the road.


In [ ]:
#4.	F1 Score: The harmonic mean of precision and recall. It balances the two metrics and is useful when you need to account for both false positives and false negatives.
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"F1 Score: {f1_score}")


- Interpretation: A higher F1 score indicates a good balance between precision and recall.


In [ ]:
#5.	Confusion Matrix: Provides a summary of true positive, false positive, true negative, and false negative counts.


cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Vehicle', 'Vehicle'], yticklabels=['No Vehicle', 'Vehicle'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


# chatgpt ouput
# cm = confusion_matrix(y_true, y_pred)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Vehicle', 'Vehicle'], yticklabels=['Non-Vehicle', 'Vehicle'])
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()


- Interpretation: The confusion matrix gives a breakdown of correct and incorrect predictions by class.
	- True Positives (TP): Correctly classified vehicles.
	- True Negatives (TN): Correctly classified non-vehicles.
	- False Positives (FP): Non-vehicles misclassified as vehicles.
	- False Negatives (FN): Vehicles misclassified as non-vehicles.


In [ ]:
#6.	Classification Report: Provides a summary of precision, recall, F1 score, and support for each class.
print(classification_report(y_true, y_pred, target_names=['No Vehicle', 'Vehicle']))

- Interpretation: The classification report gives you detailed performance metrics for each class, helping to understand which class the model struggles with.
